<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PG24_3B_Exercise_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#MySQL Tutorials <br>


1.   https://www.mysqltutorial.org/
2.   https://www.mysqltutorial.org/basic-mysql-tutorial.aspx



In [1]:
#!cat /proc/cpuinfo
#!cat /proc/meminfo
#!ls /proc/*info

#Section 1 : Getting started with MySQL

##Install MySQL in local Colab VM

In [2]:
# this will take some time and show an error.
# dont worry, just carry on
!apt update > /dev/null
!apt install -y mysql-server > /dev/null
#!ls /etc/init.d
!/etc/init.d/mysql restart
!mysql --version



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.
mysql  Ver 8.0.42-0ubuntu0.22.04.1 for Linux on x86_64 ((Ubuntu))


## Download Sample Database - Classicmodels <br>
Create Tables <br>
https://www.mysqltutorial.org/mysql-sample-database.aspx


In [3]:
#!gdown https://drive.google.com/uc?id=1Ik3QuYTd52M5qI9D4aOCfGcQ7v2AB9wx
#!wget https://sp.mysqltutorial.org/wp-content/uploads/2018/03/mysqlsampledatabase.zip
#!wget https://github.com/Praxis-QR/RDWH/raw/main/mysqlsampledatabase.zip

!wget -q https://github.com/Praxis-QR/RDWH/raw/main/data/mysqlsampledatabase.zip
!unzip mysqlsampledatabase.zip
#!cat /content/mysqlsampledatabase.sql
#!head /content/mysqlsampledatabase.sql

Archive:  mysqlsampledatabase.zip
  inflating: mysqlsampledatabase.sql  


## Loading Sample Database into MySQL server

In [4]:
#https://www.mysqltutorial.org/how-to-load-sample-database-into-mysql-database-server.aspx
!mysql --table < mysqlsampledatabase.sql
#New database classicmodels has been created
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| classicmodels      |
| information_schema |
| mysql              |
| performance_schema |
| sys                |
+--------------------+


###Check contents of sample database - Classicmodels

In [5]:
!mysql classicmodels -e 'show tables'

+-------------------------+
| Tables_in_classicmodels |
+-------------------------+
| customers               |
| employees               |
| offices                 |
| orderdetails            |
| orders                  |
| payments                |
| productlines            |
| products                |
+-------------------------+



This is what the database looks like : <br><br>
![alt text](https://www.mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)<br>

In [6]:
!mysql classicmodels -e 'select * from employees'

+----------------+-----------+-----------+-----------+---------------------------------+------------+-----------+----------------------+
| employeeNumber | lastName  | firstName | extension | email                           | officeCode | reportsTo | jobTitle             |
+----------------+-----------+-----------+-----------+---------------------------------+------------+-----------+----------------------+
|           1002 | Murphy    | Diane     | x5800     | dmurphy@classicmodelcars.com    | 1          |      NULL | President            |
|           1056 | Patterson | Mary      | x4611     | mpatterso@classicmodelcars.com  | 1          |      1002 | VP Sales             |
|           1076 | Firrelli  | Jeff      | x9273     | jfirrelli@classicmodelcars.com  | 1          |      1002 | VP Marketing         |
|           1088 | Patterson | William   | x4871     | wpatterson@classicmodelcars.com | 6          |      1056 | Sales Manager (APAC) |
|           1102 | Bondur    | Gerard    

##Install Python Client <br>


In [7]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb
from sqlalchemy import create_engine, text

In [8]:
connect_string = 'mysql://root@localhost/classicmodels'
engine = create_engine(connect_string)

In [9]:
#DBConn = MySQLdb.connect(db='praxisDB')
DBConn = engine.connect()
#df_mysql = pd.read_sql('select * from emp2;', con=con_mysql)
df_mysql = pd.read_sql(text('show tables'), con=DBConn)
print ('loaded dataframe from MySQL. records:', len(df_mysql))
DBConn.close()
df_mysql

loaded dataframe from MySQL. records: 8


,Tables_in_classicmodels
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


In [10]:
#To run any non-SELECT SQL command
def runCMD (DDL):
    #DBConn= MySQLdb.connect(db='praxisDB')
    DBConn = engine.connect()
    #myCursor = DBConn.cursor()
    try:
        #retcode = myCursor.execute(text(DDL))
        retcode = DBConn.execute(text(DDL))
        DBConn.commit()
    except :
        print('Error : return code = ',retcode)
    DBConn.close()

#To run any SELECT SQL command and return data in pandas dataframe
def runSELECT (query):
    #DBConn= MySQLdb.connect(db='praxisDB')
    DBConn = engine.connect()
    df_mysql = pd.read_sql(text(query), con=DBConn)
    #df_mysql = pd.read_sql_query(CMD, con=DBConn)
    DBConn.close()
    return df_mysql

###Check out all tables

In [11]:
#!mysql classicmodels -e 'show tables'
runSELECT('show tables')

,Tables_in_classicmodels
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


In [12]:
#runSELECT('select * from customers limit 5')
#runSELECT('select * from employees limit 5')
#runSELECT('select * from offices limit 5')
runSELECT('select * from orderdetails limit 5')
#runSELECT('select * from orders limit 5')
#runSELECT('select * from payments limit 5')
#runSELECT('select * from productlines limit 5')
#runSELECT('select * from products limit 5')

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4


#Section 1 : Querying Data

In [13]:
#code here

#Section 2 - Sorting Data

In [16]:
#code here

#Section 3 - Filtering Data

In [17]:
#code here

#Section 4 - Joining Tables

In [18]:
#code here

#Section 5 - Grouping Data

In [19]:
#code here

#Section 6 - Subqueries

In [20]:
#code here

#Section 7 - Commmon Table Expressions

In [21]:
#code here

#Section 8 - Set Operators

In [22]:
#code here

In [15]:
#Last Tested
import pandas as pd                               #
pd.to_datetime("today").strftime("%d %B %Y")      # get today's date and format it as required
                                                  # other formats https://www.geeksforgeeks.org/python-strftime-function/

'21 May 2025'

#ChronoTYM <br>
In Indic phraseology, tantra is knowledge, a yantra is a device to explore this knowledge and a mantra is the code that unlocks both the tantra and its yantra. Chrono/TYM a sci-fi trilogy that recreates this framework where the first novel Chronotantra defines the contours of a tech driven, futuristic society, the second, Chronoyantra, describes a device used to explore the genesis of this society, while the third, Chronomantra finally reveals the code, the key, that unlocks this puzzle.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjdu_ajYwBRzk5iFYlcs59ZoRMMg6rhNm4JERKspvFJoveJsnfwRlzMICiC37UOja_Bd4sbfLA1pyyBbe0mj3tUfDBXoqCzNUDw3Wux7acoJBaLkWjAvJXfa7drRY2GQRAlFxEY9O7a7raY4RKy7cA3Uxgf5QsaZQ99C9GfXxFSex_rke8ROhAn41t8zOqK/s1640/tymBanner01_3.png)